<a href="https://colab.research.google.com/github/geoburdin/characters_and_logo_on_keys/blob/main/keys_CV_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob

source_path = "/content/drive/MyDrive/keys_templates/"
images_templates = glob.glob(source_path+'*.png')

source_path = "/content/drive/MyDrive/upwork_keys/"
images_keys = glob.glob(source_path+'*.jpg')

for filepath in images_templates:
    print(filepath)

for filepath in images_keys:
    print(filepath)

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow


for image_key in images_keys:
    print(image_key)
    img = cv2.imread(image_key,0)
    blurred = cv2.GaussianBlur(img, (9,9),0)
    canny = cv2.Canny(blurred, 0, 250)
    squareKern = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    canny = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, squareKern)
    pts = np.argwhere(canny>0)
    y1,x1 = pts.min(axis=0)
    y2,x2 = pts.max(axis=0)
    cropped = img[y1:y2, x1:x2]

    for template_path in images_templates:
        template = cv2.imread(template_path, 0)
        w, h = template.shape

        im_bw = cv2.threshold(cropped, 127, 255, cv2.THRESH_BINARY)[1]
        template = cv2.threshold(template, 127, 255, cv2.THRESH_BINARY)[1]

        im_bw = cv2.GaussianBlur(im_bw, (11,11),0)
        template = cv2.GaussianBlur(template, (11,11),0)

        res = cv2.matchTemplate(im_bw, template, cv2.TM_CCOEFF_NORMED)
        threshold = .7
        loc = np.where(res >= threshold)
        for pt in zip(*loc[::-1]):  # Switch collumns and rows
            #print(template_path)
            
            cv2.putText(cropped, str(template_path[38:39]), pt, cv2.FONT_HERSHEY_SIMPLEX, 6, 0)
    cv2_imshow(cropped)
